## Approach

1. Identify available dates and temporal frequency of observations for the given collection using the GHGC API `/stac` endpoint. The collection processed in this notebook is the Earth Surface Mineral Dust Source Investigation (EMIT) methane emission plumes data product.
2. Pass the STAC item into the raster API `/stac/tilejson.json` endpoint.
3. Using `folium.Map`, visualize the plumes.
4. After the visualization, perform zonal statistics for a given polygon.

   

## About the Data

The EMIT instrument builds upon NASA’s long history of developing advanced imaging spectrometers for new science and applications. EMIT launched to the International Space Station (ISS) on July 14, 2022. The data shows high-confidence research grade methane plumes from point source emitters - updated as they are identified - in keeping with JPL Open Science and Open Data policy.

# Installing the Required Libraries
Please run the next cell to install all the required libraries to run the notebook.

In [ ]:
%pip install requests
%pip install folium
%pip install rasterstats
%pip install pystac_client

## Querying the STAC API

In [ ]:
import requests
from folium import Map, TileLayer
from pystac_client import Client

In [ ]:
# Provide STAC and RASTER API endpoints
STAC_API_URL = "http://ghg.center/api/stac"
RASTER_API_URL = "https://ghg.center/api/raster"

#Please use the collection name similar to the one used in STAC collection.

# Name of the collection for methane emission plumes. 
collection_name = "emit-ch4plume-v1"

In [ ]:
# Fetching the collection from STAC collections using appropriate endpoint.
collection = requests.get(f"{STAC_API_URL}/collections/{collection_name}").json()
collection

Examining the contents of our `collection` under the `temporal` variable, we note that data is available from August 2022 to May 2023. By looking at the `dashboard: time density`, we can see that observations are conducted daily and non-periodically (i.e., there are plumes emissions for multiple places on the same dates).

In [ ]:
# Check total number of items available
items = requests.get(f"{STAC_API_URL}/collections/{collection_name}/items?limit=500").json()["features"]
print(f"Found {len(items)} items")

In [ ]:
# Examining the first item in the collection
items[0]

Below, we are entering the minimum and maximum values to provide our upper and lower bounds in `rescale_values`.

## Exploring Methane Emission Plumes (CH₄) using the Raster API

In this notebook, we will explore global methane emission plumes from point sources. We will visualize the outputs on a map using `folium.`

In [ ]:
# To access the year value from each item more easily, this will let us query more explicity by year and month (e.g., 2020-02)
items = {item["id"]: item for item in items} 
asset_name = "ch4-plume-emissions"

In [ ]:
# Fetching the min and max values for a specific item
rescale_values = {"max":items[list(items.keys())[0]]["assets"][asset_name]["raster:bands"][0]["histogram"]["max"], "min":items[list(items.keys())[0]]["assets"][asset_name]["raster:bands"][0]["histogram"]["min"]}

Now we will pass the item id, collection name, and `rescaling_factor` to the `Raster API` endpoint. We will do this for only one item so that we can visualize the event. 

In [ ]:
color_map = "magma"
methane_plume_tile = requests.get(
    f"{RASTER_API_URL}/stac/tilejson.json?collection={items[list(items.keys())[0]]['collection']}&item={items[list(items.keys())[0]]['id']}"
    f"&assets={asset_name}"
    f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
    f"&rescale={rescale_values['min']},{rescale_values['max']}", 
).json()
methane_plume_tile

## Visualizing CH₄ Emission Plume


In [ ]:
# We will import folium to map and folium.plugins to allow side-by-side mapping
import folium
import folium.plugins

# Set initial zoom and center of map for plume Layer
map_ = folium.Map(location=(methane_plume_tile["center"][1], methane_plume_tile["center"][0]), zoom_start=13)

# December 2001
map_layer = TileLayer(
    tiles=methane_plume_tile["tiles"][0],
    attr="GHG",
    opacity=1,
)
map_layer.add_to(map_)

# visualising the map
map_



# Calculating Zonal Statistics
To perform zonal statistics, first we need to create a polygon. In this use case we will create a polygon around the plume.

In [ ]:
# Plume AOI 
methane_plume_aoi = {
    "type": "Feature",
    "properties": {},
    "geometry": {
        "coordinates":[
          [
[
          -46.450625050282454,
          -23.700764982003268
        ],
        [
          -46.41212654134425,
          -23.700764982003268
        ],
        [
          -46.41212654134425,
          -23.630816986890196
        ],
        [
          -46.450625050282454,
          -23.630816986890196
        ],
        [
          -46.450625050282454,
          -23.700764982003268
        ]
        ]],
        "type": "Polygon",
    },
}

In [ ]:
# We'll plug in the coordinates for a location
# central to the study area and a reasonable zoom level

import folium

aoi_map = Map(
    tiles="OpenStreetMap",
    location=[
        -23.630816986890196,
          -46.450625050282454
    ],
    zoom_start=14,
)

folium.GeoJson(methane_plume_aoi, name="Texas, Dallas").add_to(aoi_map)
aoi_map

In [ ]:
# Check total number of items available
items = requests.get(
    f"{STAC_API_URL}/collections/{collection_name}/items?limit=600"
).json()["features"]
print(f"Found {len(items)} items")

In [ ]:
# Explore the first item
items[0]

In [ ]:
# The bounding box should be passed to the geojson param as a geojson Feature or FeatureCollection
def generate_stats(item, geojson):
    result = requests.post(
        f"{RASTER_API_URL}/cog/statistics",
        params={"url": item["assets"][asset_name]["href"]},
        json=geojson,
    ).json()
    print(result)
    return {
        **result["properties"],
        "item_id": item["id"],
    }

In [ ]:
for item in items:
    print(item["id"])
    break

With the function above, we can generate the statistics for the area of interest.

In [ ]:
%%time
stats = [generate_stats(item, methane_plume_aoi) for item in items]

In [ ]:
stats[0]

In [ ]:
import pandas as pd


def clean_stats(stats_json) -> pd.DataFrame:
    df = pd.json_normalize(stats_json)
    df.columns = [col.replace("statistics.b1.", "") for col in df.columns]
    # df["date"] = pd.to_datetime(df["datetime"])
    return df


df = clean_stats(stats)
df.head(1)

In [ ]:
plume_tile_2 = requests.get(
    f"{RASTER_API_URL}/stac/tilejson.json?collection={items[2]['collection']}&item={items[2]['id']}"
    f"&assets={asset_name}"
    f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
    f"&rescale={rescale_values['min']},{rescale_values['max']}",
).json()
plume_tile_2

In [ ]:
# Use bbox initial zoom and map
# Set up a map located w/in event bounds
import folium

aoi_map_bbox = Map(
    tiles="OpenStreetMap",
    location=[
        -23.350,
        -46.77,
    ],
    zoom_start=13,
)

map_layer = TileLayer(
    tiles=plume_tile_2["tiles"][0],
    attr="GHG", opacity = 1
)

map_layer.add_to(aoi_map_bbox)

aoi_map_bbox

## Summary


In this notebook we have successfully explored, analyzed, and visualized the STAC collection for EMIT methane emission plumes.